# Getting financial details of the S&P 500 from Yahoo! Finance
The idea is to scrape Yahoo! Finance to get balance sheet, income statement, and cashflow statement information for each company. From this information, we can compute financial ratios to indicate financial health and profitability

In [343]:
import bs4 as bs
import urllib.request
import pandas as pd
from time import sleep

### 

## Get list of tickers in the S&P 500 from Wikipedia

In [274]:
symbols_table = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies", header=0)[0]
tickers = list(symbols_table.loc[:, "Symbol"])

### 

## Use Beautiful Soup to scrape Yahoo! Finance for each ticker
It is a good idea to space out the calls to each URL (three pages per ticker) using the sleep function, so that it doesn't lock up Yahoo! Finance. I found that using a 12 second delay per each call returned the most data. Unfortunately, this requires about five hours to retrieve all the information.

In [276]:
def yahoo_call(tickers):
    
    is_df = pd.DataFrame()
    bs_df = pd.DataFrame()
    cf_df = pd.DataFrame()
    
    for ticker in tickers:
        is_link = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
        bs_link = f'https://finance.yahoo.com/quote/{ticker}/balance-sheet?p={ticker}'
        cf_link = f'https://finance.yahoo.com/quote/{ticker}/cash-flow?p={ticker}'

        statements_list = [is_link,bs_link,cf_link]

        headers = []
        temp_list = []
        label_list = []
        final = []
        index = 0

        df_lists = list()

        for link in statements_list:
            sleep(12)
            print(ticker + ': ' + link)

            try:
                source = urllib.request.urlopen(link).read()
                soup = bs.BeautifulSoup(source,'html')

                features = soup.find_all('div', class_='D(tbr)')

                for item in features[0].find_all('div', class_='D(ib)'):
                    headers.append(item.text)

                while index <= len(features)-1:
                    temp = features[index].find_all('div', class_='D(tbc)')
                    for line in temp:
                        temp_list.append(line.text)
                    final.append(temp_list)
                    temp_list = []
                    index+=1

                df = pd.DataFrame(final[1:])
                df.columns = headers


                def convert_to_numeric(column):

                    first_col = [i.replace(',','') for i in column]
                    second_col = [i.replace('-','') for i in first_col]
                    final_col = pd.to_numeric(second_col)

                    return final_col

                for column in headers[1:]:
                    df[column] = convert_to_numeric(df[column])

                df = df.iloc[:,0:2]
                df.rename(columns = {df.columns[1]: ticker}, inplace = True)
                final_df = df.fillna('-')
                final_df.set_index(['Breakdown'], inplace = True)
                df_lists.append(final_df)

                headers = []
                temp_list = []
                label_list = []
                final = []
                index = 0

                if link == is_link:
                    if len(is_df) == 0:
                        is_df = df_lists[0]
                    else:
                        is_df = is_df.join(df_lists[0])
                elif link == bs_link:
                    if len(bs_df) == 0:
                        bs_df = df_lists[1]
                    else:
                        bs_df = bs_df.join(df_lists[1])
                elif link == cf_link:
                    if len(cf_df) == 0:
                        cf_df = df_lists[2]
                    else:
                        cf_df = cf_df.join(df_lists[2])
            except:
                print('Exception: ' + ticker + ' - ' + link)
                pass
            
    return is_df, bs_df, cf_df

In [277]:
income_statements, balance_sheets, cashflow_statements = yahoo_call(tickers)

MMM: https://finance.yahoo.com/quote/MMM/financials?p=MMM
MMM: https://finance.yahoo.com/quote/MMM/balance-sheet?p=MMM
MMM: https://finance.yahoo.com/quote/MMM/cash-flow?p=MMM
ABT: https://finance.yahoo.com/quote/ABT/financials?p=ABT
ABT: https://finance.yahoo.com/quote/ABT/balance-sheet?p=ABT
ABT: https://finance.yahoo.com/quote/ABT/cash-flow?p=ABT
ABBV: https://finance.yahoo.com/quote/ABBV/financials?p=ABBV
ABBV: https://finance.yahoo.com/quote/ABBV/balance-sheet?p=ABBV
ABBV: https://finance.yahoo.com/quote/ABBV/cash-flow?p=ABBV
ABMD: https://finance.yahoo.com/quote/ABMD/financials?p=ABMD
ABMD: https://finance.yahoo.com/quote/ABMD/balance-sheet?p=ABMD
ABMD: https://finance.yahoo.com/quote/ABMD/cash-flow?p=ABMD
ACN: https://finance.yahoo.com/quote/ACN/financials?p=ACN
ACN: https://finance.yahoo.com/quote/ACN/balance-sheet?p=ACN
ACN: https://finance.yahoo.com/quote/ACN/cash-flow?p=ACN
ATVI: https://finance.yahoo.com/quote/ATVI/financials?p=ATVI
ATVI: https://finance.yahoo.com/quote/ATV

### 

## Inspect each of the financial statements that is returned

In [278]:
income_statements

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Breakdown,,,,,,,,,,,,,,,,,,,,,
Total Revenue,31712000,32221000,34057000,802857,44327039,6452000,12436297,8646000,9623982,9531000,...,5913269,11377000,7536000,2972876,5249000,5603000,4332000,7790500,2784000,6542000
Cost of Revenue,16542000,13944000,7687000,154084,30350881,2036000,1745720,4801000,5413839,7184000,...,3798131,6461000,4593000,925371,3154000,2908000,2403000,2186300,NaN,1986000
Gross Profit,15170000,18277000,26370000,648773,13976158,4416000,10690577,3845000,4210143,2347000,...,2115138,4916000,2943000,2047505,2046000,2695000,1929000,5604200,NaN,4556000
Operating Expense,8922000,13963000,12564000,425059,7462514,2669000,6698456,2698000,3570405,179000,...,1694679,2776000,2296000,1300777,1349000,1024000,1283000,4489900,NaN,2300000
Operating Income,6248000,4314000,13806000,223714,6513644,1747000,3992121,1147000,639738,2168000,...,420459,2140000,647000,746728,697000,1671000,646000,1114300,NaN,2256000
Net Non Operating Interest Income Expense,472000,512000,1612000,NaN,36260,16000,69893,41000,41944,729000,...,424562,780000,74000,17426,62000,543000,73000,219800,NaN,212000
Other Income Expense,437000,18000,3214000,46606,224427,61000,13513,172000,4282,1425000,...,43548,148000,88000,15705,219000,63000,50000,784300,NaN,31000
Pretax Income,6213000,3820000,8980000,270320,6774331,1670000,3935741,934000,593512,14000,...,47651,1508000,485000,713597,416000,1065000,523000,110200,548000,2013000
Tax Provision,1244000,458000,544000,62504,1589018,109000,74283,57000,140388,105000,...,250955,31000,109000,101676,15000,5000,49000,266000,110000,351000


In [279]:
balance_sheets

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Breakdown,,,,,,,,,,,,,,,,,,,,,
Total Assets,44659000,67887000,89115000,1216462,37078593,19845000,20762400,6028000,11248525,33648000,...,13871281,50448000,15047000,4693334,7710000,5231000,4711000,24638700,69172000,11545000
Total Liabilities Net Minority Interest,34533000,36586000,97287000,150996,19579420,7040000,10232245,3201000,7699444,28419000,...,12329809,37209000,9239000,2378285,4743000,13247000,2872000,12245900,61819000,8837000
Total Equity Gross Minority Interest,10126000,31301000,8172000,1065466,17499173,12805000,10530155,2827000,3549081,5229000,...,1541472,13239000,5808000,2315049,2967000,8016000,1839000,12392800,7353000,2708000
Total Capitalization,27581000,47749000,54803000,1065466,17054588,15480000,11519079,3313000,4296401,21301000,...,11823028,30646000,9034000,3062159,4997000,2045000,2919000,19103100,9072000,8655000
Common Stock Equity,10063000,31088000,8172000,1065466,17000536,12805000,10530155,2827000,3549081,2996000,...,1743045,13239000,5587000,2315049,2957000,8016000,1839000,12388100,6787000,2708000
Capital Lease Obligations,965000,755000,nan,-,3423641,NaN,nan,242000,2017159,nan,...,159182,1549000,354000,11109,61000,784000,129000,nan,4000,164000
Net Tangible Assets,9760000,9132000,42425000,1018584,8261498,2456000,1881609,2328000,1847085,1468000,...,1596631,13239000,1404000,1374070,1056000,8790000,1058000,4469200,6338000,1774000
Working Capital,3749000,4804000,33934000,503978,5087166,4377000,1696013,2238000,1217841,135000,...,873453,1455000,2705000,1823460,949000,14000,209000,1282400,nan,2942000
Invested Capital,30376000,49227000,58556000,1065466,17062408,15480000,14668422,3313000,4296401,23169000,...,12146904,31943000,9869000,3561419,5273000,2469000,3116000,20603100,9506000,9155000


In [280]:
cashflow_statements

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Breakdown,,,,,,,,,,,,,,,,,,,,,
Operating Cash Flow,7936000,6519000,14122000,285001,8215152,1529000,5321813,959000,822958,2778000,...,460712,2880000,751000,1161980,839000,1285000,796000,1753100,899000,1990000
Investing Cash Flow,5620000,1973000,494000,168758,1894519,45000,451584,684000,500110,2866000,...,894877,4235000,1993000,1138278,202000,303000,648000,745000,8203000,608000
Financing Cash Flow,5949000,4177000,21669000,61421,4049092,254000,3314044,136000,74136,704000,...,1480118,1376000,427000,456739,177000,474000,114000,848100,7084000,470000
End Cash Position,4098000,4460000,41182000,205101,8415330,5930000,3765185,1300000,1144703,2404000,...,2872746,885000,3296000,1170776,724000,1344000,67000,2443000,576000,3667000
Income Tax Paid Supplemental Data,-,-,-,28590,1360030,nan,320478,-,-,325000,...,-,NaN,NaN,18967,107000,nan,138000,NaN,202000,430000
Interest Paid Supplemental Data,-,-,-,-,28493,nan,115075,-,-,883000,...,390147,736000,NaN,39852,77000,nan,42000,NaN,250000,244000
Capital Expenditure,1617000,1932000,570000,38951,599132,117000,410479,262000,500558,2152000,...,897330,4888000,77000,97239,226000,186000,66000,745000,146000,475000
Issuance of Capital Stock,435000,NaN,NaN,NaN,955308,101000,270932,-,3204,NaN,...,-,6000,-,NaN,nan,nan,nan,NaN,11000,NaN
Issuance of Debt,1910000,3122000,34482000,NaN,nan,1994000,3144000,NaN,998240,10752000,...,5112806,3923000,1857000,744427,281000,1785000,433000,2046300,495000,-


In [281]:
income_statements.to_csv('income_statements.csv')
balance_sheets.to_csv('balance_sheets.csv')
cashflow_statements.to_csv('cashflow_statements.csv')

### 

## Compute financial ratios

To capture indicators of financial health and profitability, we can use ratios from the information that was returned. These three ratios are the most common ratios that are used, but others can be computed.
- Total debt ratio = Total debt / Total assets
- Profit margin = Net income / Total sales
- Debt-to-equity = Total debt / Total common equity

In [334]:
pd.set_option("display.precision", 8)

### 

### Use balance sheet information to compute debt ratios

In [282]:
debt_ratios = balance_sheets.T

In [283]:
debt_ratios

Breakdown,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Treasury Shares Number
MMM,44659000,34533000,10126000,27581000,10063000,965000,9760000,3749000,30376000,9760000,21278000,17960000,944033,575185,368848
ABT,67887000,36586000,31301000,47749000,31088000,755000,9132000,4804000,49227000,9132000,18894000,14279000,1976855,1762503,214352
ABBV,89115000,97287000,8172000,54803000,8172000,NaN,42425000,33934000,58556000,42425000,66728000,26804000,1781583,1478911,302671
ABMD,1216462,150996,1065466,1065466,1065466,-,1018584,503978,1065466,1018584,-,NaN,47542,45009,2533
ACN,37078593,19579420,17499173,17054588,17000536,3423641,8261498,5087166,17062408,8261498,3485513,NaN,659076,634693,1802161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,5231000,13247000,8016000,2045000,8016000,784000,8790000,14000,2469000,8790000,11269000,9880000,300000,300000,NaN
ZBRA,4711000,2872000,1839000,2919000,1839000,129000,1058000,209000,3116000,1058000,1406000,1247000,72152,54003,18149
ZBH,24638700,12245900,12392800,19103100,12388100,NaN,4469200,1282400,20603100,4469200,8215000,7597100,309900,206000,103900
ZION,69172000,61819000,7353000,9072000,6787000,4000,6338000,NaN,9506000,5772000,2723000,1271000,165057,165057,NaN


In [295]:
debt_ratios['Total Assets'] = debt_ratios['Total Assets'].replace('-',0)
debt_ratios['Total Debt'] = debt_ratios['Total Debt'].replace('-',0)
debt_ratios['Common Stock Equity'] = debt_ratios['Common Stock Equity'].replace('-',0)

In [335]:
debt_ratios['debt_to_assets'] = debt_ratios['Total Debt'].astype(float) / debt_ratios['Total Assets'].astype(float)

In [336]:
debt_ratios['debt_to_equity'] = debt_ratios['Total Debt'].astype(float) / debt_ratios['Common Stock Equity'].astype(float)

### 

### Use income statement information to compute profitability ratio

In [298]:
profitability = income_statements.T

In [299]:
profitability

Breakdown,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,Net Income Common Stockholders,...,EBIT,EBITDA,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Total Unusual Items Excluding Goodwill,Total Unusual Items,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
MMM,31712000,16542000,15170000,8922000,6248000,472000,437000,6213000,1244000,4964000,...,6725000,8601000,16542000,1876000,4964000,389000,389000,8212000,0,77888
ABT,32221000,13944000,18277000,13963000,4314000,512000,18000,3820000,458000,3382000,...,4394000,7611000,12834000,3217000,3362000,5000,5000,7606000,0,599
ABBV,34057000,7687000,26370000,12564000,13806000,1612000,3214000,8980000,544000,8394000,...,10940000,13013000,7687000,2073000,8436000,271000,271000,13284000,0,16417
ABMD,802857,154084,648773,425059,223714,NaN,46606,270320,62504,207816,...,223714,246229,154084,22515,207816,10405,10405,235824,0,2406
ACN,44327039,30350881,13976158,7462514,6513644,36260,224427,6774331,1589018,5107839,...,6807402,8580526,30350881,1773124,4678209,-,-,8580526,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,5603000,2908000,2695000,1024000,1671000,543000,63000,1065000,5000,1060000,...,1608000,1736000,2908000,128000,1060000,54000,54000,1790000,0,254
ZBRA,4332000,2403000,1929000,1283000,646000,73000,50000,523000,49000,474000,...,596000,735000,2335000,139000,474000,57000,57000,792000,0,5340
ZBH,7790500,2186300,5604200,4489900,1114300,219800,784300,110200,266000,377000,...,330000,1341400,1763400,1011400,377000,783000,783000,2124400,0,211410
ZION,2784000,NaN,NaN,NaN,NaN,NaN,NaN,548000,110000,400000,...,NaN,NaN,NaN,120000,438000,NaN,NaN,NaN,0,0


In [337]:
profitability['profit_margin'] = profitability['Net Income from Continuing & Discontinued Operation'].astype(float) / profitability['Total Revenue'].astype(float)

### 

### Consolidate all financial ratios into one DataFrame

In [338]:
financial_ratios = pd.DataFrame(tickers)

In [339]:
financial_ratios.rename(columns = {0:'tickers'}, inplace = True)
financial_ratios.set_index('tickers', inplace = True)

In [340]:
financial_ratios = financial_ratios.join(debt_ratios[['debt_to_assets','debt_to_equity']])

In [341]:
financial_ratios = financial_ratios.join(profitability['profit_margin'])

In [344]:
financial_ratios

,debt_to_assets,debt_to_equity,profit_margin
tickers,,,
MMM,0.47645491,2.11447878,0.15653380
ABT,0.27831544,0.60775862,0.10496260
ABBV,0.74878528,8.16544298,0.24770238
ABMD,0.00000000,0.00000000,0.25884560
ACN,0.09400338,0.20502371,0.11523077
...,...,...,...
YUM,2.15427261,1.40581337,0.18918437
ZBRA,0.29845044,0.76454595,0.10941828
ZBH,0.33341857,0.66313640,0.04839227


In [349]:
financial_ratios.to_csv('financial_ratios.csv')